<hr />

### NoteBookの見方
コード以外の情報 
<span >白/黒での記載は速習Symbol</span>  
<span style="color:red">赤色での記載は補足情報</span>  
<span >コード内で🌟マークがある場所は自分の情報に書き換えが必要</span>

<hr />

# 環境構築

## 1.Symbol SDKの読み込み </span>

In [ ]:
(script = document.createElement("script")).src = "https://xembook.github.io/nem2-browserify/symbol-sdk-pack-2.0.3.js";
document.getElementsByTagName("head")[0].appendChild(script);

## 2.Symbol用の共通設定

In [ ]:
NODE = 'https://sym-test-03.opening-line.jp:3001';
sym = require("/node_modules/symbol-sdk");
repo = new sym.RepositoryFactoryHttp(NODE);
txRepo = repo.createTransactionRepository();
mosaicRepo = repo.createMosaicRepository();
accountRepo = repo.createAccountRepository();
(async () => {
  networkType = await repo.getNetworkType().toPromise();
  generationHash = await repo.getGenerationHash().toPromise();
  epochAdjustment = await repo.getEpochAdjustment().toPromise();
})();

function clog(signedTx){
    console.log(NODE + "/transactionStatus/" + signedTx.hash);
    console.log(NODE + "/transactions/confirmed/" + signedTx.hash);
    console.log("https://symbol.fyi/transactions/" + signedTx.hash);
    console.log("https://testnet.symbol.fyi/transactions/" + signedTx.hash);
}

## 3.各種アカウントの復元

<span style="color:red">
※各種アカウントがない場合は3-1へ
</span>

In [ ]:
alice = sym.Account.createFromPrivateKey("🌟privateKey🌟", networkType);
main_bot = sym.Account.createFromPrivateKey("🌟privateKey🌟", networkType);
sub_bot = sym.Account.createFromPrivateKey("🌟privateKey🌟", networkType);
qr = sym.Account.createFromPrivateKey("🌟privateKey🌟", networkType);
bob = sym.Account.createFromPrivateKey("🌟privateKey🌟", networkType);

console.log(alice.address.plain());
console.log(main_bot.address.plain());
console.log(sub_bot.address.plain());
console.log(qr.address.plain());
console.log(bob.address.plain());

## 3-1.alice,main_bot,sub_bot,qr,bobアカウントの生成
<span style="color:red">
※各種アカウントがない場合のみ
</span>

In [ ]:
alice = sym.Account.generateNewAccount(networkType);
main_bot = sym.Account.generateNewAccount(networkType);
sub_bot = sym.Account.generateNewAccount(networkType);
qr = sym.Account.generateNewAccount(networkType);
bob = sym.Account.generateNewAccount(networkType);

console.log(alice.address.plain());
console.log(main_bot.address.plain());
console.log(sub_bot.address.plain());
console.log(qr.address.plain());
console.log(bob.address.plain());

now = new Date().getTime();

data = 'alice: ' + alice.privateKey + "\r\n";
data += 'main_bot: ' + main_bot.privateKey + "\r\n";
data += 'sub_bot: ' + sub_bot.privateKey + "\r\n";
data += 'qr: ' + qr.privateKey + "\r\n";
data += 'bob: ' + bob.privateKey + "\r\n";
if ( ~navigator.userAgent.indexOf("Windows") ) {
  data = data.replace(/\n/g, "\r\n").replace(/\r\r/g, "\r")
}
const blob = new Blob([data], {type: "text/plain"});
const link = document.createElement("a");
link.href = URL.createObjectURL(blob);
link.download = `privatekey-${now}.txt`;
link.click();

## 4.各アカウントへ残高補充

テストネットの場合はFAUCETでアカウントにXYMを補給しておきます。<br />
既に残高がある場合は不要。
- Faucet
    - https://testnet.symbol.tools/

##### URL出力


In [ ]:
console.log("https://testnet.symbol.tools/?recipient=" + alice.address.plain() +"&amount=500");
console.log("https://testnet.symbol.tools/?recipient=" + qr.address.plain() +"&amount=50");
console.log("https://testnet.symbol.tools/?recipient=" + main_bot.address.plain() +"&amount=1");
console.log("https://testnet.symbol.tools/?recipient=" + sub_bot.address.plain() +"&amount=1");


## 5.aliceアカウントのマルチシグの登録
<span style="color:red">
※既にマルチシグ化してある場合は不要
</span>

In [ ]:
multisigTx = sym.MultisigAccountModificationTransaction.create(
  undefined, 
  3, //minApproval:承認のために必要な最小署名者数増分
  2, //minRemoval:除名のために必要な最小署名者数増分
  [
      qr.address,
      main_bot.address,
      sub_bot.address
  ], //追加対象アドレスリスト
  [],//除名対象アドレスリスト
  networkType
);
aggregateTx = sym.AggregateTransaction.createComplete(
  sym.Deadline.create(epochAdjustment),
  [//マルチシグ化したいアカウントの公開鍵を指定
    multisigTx.toAggregate(alice.publicAccount),
  ],
  networkType,[]
).setMaxFeeForAggregate(100, 3); // 第二引数に連署者の数:2
signedTx =  aggregateTx.signTransactionWithCosignatories(
  alice, //マルチシグ化したいアカウント
  [qr, main_bot, sub_bot], //追加・除外対象として指定したアカウント
  generationHash,
);
await txRepo.announce(signedTx).toPromise();



In [ ]:
hash = signedTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

## 5-2.確認

マルチシグ化したアカウントの確認

In [ ]:
msigRepo = repo.createMultisigRepository();

multisigInfo = await msigRepo.getMultisigAccountInfo(alice.address).toPromise();
console.log(multisigInfo);

## 6.Botアカウントの起動

main_botとsub_botの書名要求を検知するようにリスナーを登録します。<br />
署名要求があればbotアカウントは自動的に署名を行います。<br />

<span style="color:red">

### notebook形式では実行できないため、[こちら](multisig_payment.md)の方法を参照してください。

</span>

## 7.マルチシグ署名

マルチシグ化したアカウントからモザイクを送信します。送信先はbobです。

### アグリゲートボンデッドトランザクションで送信

アグリゲートボンデッドトランザクションの場合は連署者を指定せずにアナウンスできます。<br>
事前にハッシュロックでトランザクションを留め置きしておくことを宣言しておき、連署者がネットワーク上に留め置きされたトランザクションに追加署名することで完成となります。

In [ ]:
send_amount = 1; //1XYM
tx = sym.TransferTransaction.create(
  undefined,
  bob.address, //bobへの送信
  [new sym.Mosaic(new sym.NamespaceId("symbol.xym"),sym.UInt64.fromUint(send_amount * 1000000))],
  sym.PlainMessage.create('payment'),
  networkType
);
aggregateTx = sym.AggregateTransaction.createBonded(
  sym.Deadline.create(epochAdjustment),
    [ //マルチシグ化したアカウントの公開鍵を指定
      tx.toAggregate(alice.publicAccount)
    ],
  networkType,[],
).setMaxFeeForAggregate(100, 2); // 第二引数に連署者の数:2
//qrアカウントで書名
signedAggregateTx = qr.sign(aggregateTx, generationHash);
//ハッシュロックTXを作成
hashLockTx = sym.HashLockTransaction.create(
sym.Deadline.create(epochAdjustment),
new sym.Mosaic(new sym.NamespaceId("symbol.xym"),sym.UInt64.fromUint(10 * 1000000)), //固定値:10XYM
sym.UInt64.fromUint(480),
signedAggregateTx,
networkType
).setMaxFee(100);
//qrアカウントで署名（10XYMはqrアカウントが担保する）
signedLockTx = qr.sign(hashLockTx, generationHash);
//ハッシュロックTXをアナウンス
await txRepo.announce(signedLockTx).toPromise();

### ハッシュロックがconfirmedになっていることを確認

In [ ]:
hash = signedLockTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);

In [ ]:
//ハッシュロックの承認を確認した後、ボンデッドTXをアナウンス
await txRepo.announceAggregateBonded(signedAggregateTx).toPromise();

# これ以降は 6.Botアカウントの起動 の起動を行っている場合不要です。

一応、何かしらの原因でトランザクションに署名できないときのために記載しておきます。

In [ ]:
hash = signedAggregateTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Partial).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

ボンデッドトランザクションがノードに取り込まれるとパーシャル署名状態となるので、8.ロックで紹介した連署を使用して、マルチシグアカウントで連署します。 連署をサポートするウォレットで承認することもできます。

main_botで署名

In [ ]:
targetHash = hash; //本来は自分宛のPartialトランザクションを確認して、その上でハッシュ値を確認し指定するが、今回は変数からハッシュ値を取り出している
txInfo = await txRepo.getTransaction(targetHash,sym.TransactionGroup.Partial).toPromise(); //ハッシュ値でトランザクションを検索
cosignatureTx = sym.CosignatureTransaction.create(txInfo); //連署用のトランザクションを作成
signedCosTx = main_bot.signCosignatureTransaction(cosignatureTx);
await txRepo.announceAggregateBondedCosignature(signedCosTx).toPromise(); //ブロックチェーンにアナウンス

In [ ]:
hash = signedAggregateTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Partial).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

sub_botで署名

In [ ]:
targetHash = hash; //本来は自分宛のPartialトランザクションを確認して、その上でハッシュ値を確認し指定するが、今回は変数からハッシュ値を取り出している
txInfo = await txRepo.getTransaction(targetHash,sym.TransactionGroup.Partial).toPromise(); //ハッシュ値でトランザクションを検索
cosignatureTx = sym.CosignatureTransaction.create(txInfo); //連署用のトランザクションを作成
signedCosTx = sub_bot.signCosignatureTransaction(cosignatureTx);
await txRepo.announceAggregateBondedCosignature(signedCosTx).toPromise(); //ブロックチェーンにアナウンス

# 最後に

トランザクションがconfirmedになれば送信完了です！

In [ ]:
hash = signedAggregateTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加